In [ ]:
pip install openai num2words matplotlib plotly scipy scikit-learn pandas tiktoken

pip install AzureOpenAI

import openai
client = openai.AzureOpenAI(
  api_key = "보안",  
  api_version = "보안",
  azure_endpoint = "보안"
)

import os
import re
import requests
import sys
from num2words import num2words
import os
import pandas as pd
import numpy as np
import tiktoken
from openai import AzureOpenAI

df=pd.read_csv(os.path.join(os.getcwd(),'2nd_project_data_final 1.csv')) # This assumes that you have placed the bill_sum_data.csv in the same directory you are running Jupyter Notebooks
df


df_inde = df[['id','name','nameHanja','movementFamily','orders','addressBirth','aliases','bornDied','references','content','activities','engagedOrganizations','isForeigner','image_url'
]]

df_inde



In [ ]:
import pandas as pd
import re

# 경고 끄기 (SettingWithCopyWarning 방지)
pd.options.mode.chained_assignment = None

# 예외를 포함한 텍스트 정규화 함수
def normalize_text(s, sep_token=" \n "):
    if not isinstance(s, str):
        return ""  # 문자열이 아닌 경우 빈 문자열로 처리

    s = re.sub(r'\s+', ' ', s).strip()         # 다중 공백 제거
    s = re.sub(r"\.\s*,", "", s)               # '. ,' 패턴 제거
    s = s.replace("..", ".")                   # '..' → '.'
    s = s.replace(". .", ".")                  # '. .' → '.'
    s = s.replace("\n", "")                    # 줄바꿈 제거
    s = s.strip()
    return s

# DataFrame 불러온 후: text 컬럼 전처리
# 예: df_inde = pd.read_csv("독립운동가.csv")
# 전처리 전에 결측치 먼저 처리
df_inde['content'] = df_inde['content'].fillna("")        # NaN → 빈 문자열
df_inde['content'] = df_inde['content'].apply(normalize_text)  # 전처리 적용



tokenizer = tiktoken.get_encoding("cl100k_base")
df_inde['n_tokens'] = df_inde["content"].apply(lambda x: len(tokenizer.encode(x)))
df_bills = df_inde[df_inde.n_tokens<8192]
len(df_bills)


df_inde = df_inde[df_inde.n_tokens < 8192]
# df_inde.sort_values(by="n_tokens", ascending=False).head(10)
df_inde


sample_encode = tokenizer.encode(df_inde.content[0]) 
decode = tokenizer.decode_tokens_bytes(sample_encode)
decode

len(decode)

In [ ]:
import time
from openai import AzureOpenAI
import pandas as pd

# 1. Azure OpenAI 클라이언트 설정
client = AzureOpenAI (
    api_key = "보안",  
    api_version = "보안",
    azure_endpoint = "보안"
)

deployment_name = "text-embedding-ada-002"  # 예: "embedding-v2"

# 2. 배치 임베딩 함수
def generate_embeddings_batch(text_list, model=deployment_name):
    try:
        response = client.embeddings.create(input=text_list, model=model)
        return [item.embedding for item in response.data]
    except Exception as e:
        print("❌ 배치 임베딩 실패:", e)
        return [None] * len(text_list)  # 실패한 경우 None으로 채움

# 3. 데이터 준비
df_inde = df_inde[df_inde['content'].apply(lambda x: isinstance(x, str) and x.strip() != "")]
df_inde = df_inde[df_inde['n_tokens'] < 8192]
df_inde = df_inde.reset_index(drop=True)

# 4. 배치 처리 루프
batch_size = 100
all_embeddings = []

for i in range(0, len(df_inde), batch_size):
    batch_texts = df_inde['content'][i:i+batch_size].tolist()
    print(f"🔄 {i} ~ {i+len(batch_texts)-1} 처리 중...")
    
    batch_embeddings = generate_embeddings_batch(batch_texts)
    all_embeddings.extend(batch_embeddings)
    
    time.sleep(1.2)  # 속도 조절 (rate limit 방지)

# 5. 결과 붙이기
df_inde['ada_v2'] = all_embeddings

# ✅ 완료 메시지 출력
print(f"\n🎉 총 {len(df_inde)}건 임베딩 완료되었습니다!")



df_inde.to_csv("임베딩_완료2.csv", index=False)